In [1]:
!pip install pyspark
!pip install praw
!pip3 install iexfinance

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 19.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f56cc4c0c07794fec954f44b098c74ad70a198ea0574dafc2344a9b7bdae4a3b
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
     |████████████████████████████████| 163kB 5.0MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 61kB 2.7MB/s 


In [2]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
#from iexfinance.stocks import Stock
#from iexfinance.stocks import get_historical_data
import os
# tools
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta
import string
# Set IEX Finance API Token for Sandbox test mode
os.environ['IEX_API_VERSION'] = 'iexcloud-sandbox'
os.environ['IEX_TOKEN'] = ''

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark
spark = init_spark()

In [3]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [4]:
"""
Function to make an HTTP request to the Pushshift API.
- max_retry: Nb of times the request is re-tried if failure occurs.
- returns: Python object with the content of the request. (index 'data' property)
"""
def get_request(uri, max_retry = 5):
  def get(uri):
    response = requests.get(uri)
    assert response.status_code == 200
    return json.loads(response.content)
  # Retry if request call failed
  retry = 1
  while retry < max_retry:
    try:
      response = get(uri)
      return response
    except:
      #print(f"[{retry}] Request failed, re-trying...")
      # wait 1 second before retry
      time.sleep(1)
      retry += 1

In [5]:
# Testing get_request() with test uri. Should return a non-empty Python object.
obj = get_request("https://httpbin.org/get")
print(obj['url'])
# Returning posts from wallstreetbets. The posts are in the "data" property of the response.
# Use this to check format: https://jsonformatter.curiousconcept.com/#
# obj2 = get_request('https://api.pushshift.io/reddit/search/submission?subreddit=wallstreetbets')
# print(obj2)

https://httpbin.org/get


In [6]:
"""
Gets the all the posts from a given subreddit in the specific time range.
- subreddit: name of subreddit
- begin: timestamp (in unix) of start date
- end: timestamp (in unix) of end date
- returns: list of all the posts in the time interval as objects. {id="..", title="...", etc...}
"""
def get_posts(subreddit, begin, end):
  # Max size of each Pushshift API request is 100 posts.
  SIZE = 100
  PUSHSHIFT_URI = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
  nb_requests_made = 1
  loading_string = "LOADING %: ["
  print("LOADING %: [", end=" ")

  # filter the posts data
  def filter_posts(uri, begin, end):
    full_posts = get_request(uri.format(subreddit, begin, end, SIZE))
    if full_posts is None:
      raise ValueError("Response is empty or none.")

    posts = []
    for post in full_posts['data']:
      try:
        if post['link_flair_text'] and post['link_flair_css_class'] and post['selftext'] and post['selftext'] != "[removed]" and post['selftext'] != "[deleted]":
          posts.append({\
            'id': post['id'],\
            'title': post['title'].translate(str.maketrans('', '', string.punctuation)),\
            'score': post['score'],\
            'upvote_ratio': post['upvote_ratio'],\
            'author': post['author'],\
            'created_utc': post['created_utc'],\
            'flair': post['link_flair_text'],\
            'flaircss': post['link_flair_css_class'],\
            'num_comments': post['num_comments'],\
            'text': post['selftext'].translate(str.maketrans('', '', string.punctuation)),\
            'url': post['url']\
          })
      except:
        pass
    # get timestamp of last post
    last_timestamp = full_posts['data'][-1]['created_utc']
    posts_amount = len(full_posts['data'])
          
    #return list(filtered)
    return [posts, last_timestamp, posts_amount]

  posts_etc = filter_posts(PUSHSHIFT_URI, begin, end)
  posts = posts_etc[0]
  last_timestamp = posts_etc[1]
  posts_amount = posts_etc[2]
  # If reached limit of 100 posts retrieved, make request again until 'end' time.
  while posts_amount == SIZE:
    # Timestamp of the last post we previously retrieved
    new_begin = last_timestamp - 10
    more_posts_etc = filter_posts(PUSHSHIFT_URI, new_begin, end)
    last_timestamp = more_posts_etc[1]
    posts_amount = more_posts_etc[2]
    posts.extend(more_posts_etc[0])
    nb_requests_made += 1
    printProgressBar((last_timestamp-begin)//10000 + 1, (end-begin)//10000)
    
  print("Number of requests made: ", nb_requests_made)
  return posts

In [7]:
"""
Retrieve posts
- nb_days_from_today: number of days from today you want to retrieve
- return: lists of all posts in time interval
"""

def retrieve(nb_days_from_today):
  end = math.ceil(datetime.utcnow().timestamp())
  begin = math.ceil((datetime.utcnow() - timedelta(days=nb_days_from_today)).timestamp())
  print("Timestamps: ", begin, end)
  posts = get_posts('wallstreetbets', begin, end)

  unique_posts = np.unique([post['id'] for post in posts])
  print("Size: ", len(posts))
  print("Size of uniques: ", len(unique_posts))
  print("Example posts: ", posts[:5])
  return posts

# all posts!
posts = retrieve(1)

Timestamps:  1616902145 1616988545
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Number of requests made:  13
Size:  164
Size of uniques:  162
Example posts:  [{'id': 'metnlc', 'title': 'Tesla Marketing Idea ELON I HOPE YOU SEE THIS', 'score': 1, 'upvote_ratio': 1.0, 'author': 'corona4all', 'created_utc': 1616902459, 'flair': 'Meme', 'flaircss': 'meme', 'num_comments': 0, 'text': 'I was thinking of ways to get this Daddy Elons TSLA Stock to MARS\n\nThought this maybe an amazing idea by monetizing on this Cargo Ship Evergreen drama and promoting the Cybertruck that has more angles than a isoceles triangle\n\nBon Apetit\n\nhttpsimgurcomWFFrcuAhttpsimgurcomWFFrcuA', 'url': 'https://www.reddit.com/r/wallstreetbets/comments/metnlc/tesla_marketing_idea_elon_i_hope_you_see_this/'}, {'id': 'metp1b', 'title': 'When I see US retailer investors talk about taxes', 'score': 1, 'upvote_ratio': 1.0, 'author': 'U-Copy', 'created_utc': 1

In [9]:
"""
OUTPUT TO CSVVVVVVV

using RDD and DF
"""
posts_rdd = spark.sparkContext.parallelize(posts)
posts_rdd = posts_rdd.map(lambda post: Row(id=post['id'], title=post['title'], flair=str(post['flair']), score=post['score'], upvote_ratio=post['upvote_ratio'], author=str(post['author']), num_comments=post['num_comments'], text=post['text'], created=post['created_utc'], url=post['url']))
# eliminate duplicates
posts_rdd = posts_rdd.distinct()
posts_df = spark.createDataFrame(posts_rdd)
print(posts_df.count())
posts_df.toPandas()

# write to csv
#posts_df.coalesce(1).write.csv('30daysredditposts.csv', header=True)

162


,id,title,flair,score,upvote_ratio,author,num_comments,text,created,url
0,metnlc,Tesla Marketing Idea ELON I HOPE YOU SEE THIS,Meme,1,1.00,corona4all,0,I was thinking of ways to get this Daddy Elons...,1616902459,https://www.reddit.com/r/wallstreetbets/commen...
1,metp1b,When I see US retailer investors talk about taxes,Meme,1,1.00,U-Copy,2,ampx200B\n\nhttpspreviewredditt6cyx1l0wop61jpg...,1616902616,https://www.reddit.com/r/wallstreetbets/commen...
2,mettu0,When I see post US retail investors talking ab...,Meme,1,1.00,U-Copy,1,ampx200B\n\nhttpspreviewreddit5dlemxkpxop61jpg...,1616903167,https://www.reddit.com/r/wallstreetbets/commen...
3,metz4t,BUCKLE UP we just saw a hedge fund die this w...,DD,1,1.00,boneywankenobi,632,Hooo boy I just figured out something amazing ...,1616903745,https://www.reddit.com/r/wallstreetbets/commen...
4,meu4mh,Who like Games,Discussion,1,1.00,r_ventura_23,0,I play a game called Kings Age The US server ...,1616904345,https://www.reddit.com/r/wallstreetbets/commen...
...,...,...,...,...,...,...,...,...,...,...
157,mfgfh8,Ozsc🤑🤑🤑,Gain,1,1.00,Fine_Strength3137,1,Green day tomorrow 🚀🚀🚀,1616987305,https://www.reddit.com/r/wallstreetbets/commen...
158,mfggcd,BLDV IS ABOUT TO TAKE OFF 🚀🚀,News,1,1.00,Beautiful_Peanut_721,1,Are you all in yet,1616987396,https://www.reddit.com/r/wallstreetbets/commen...
159,mfggra,Tomorrows Market,Discussion,1,1.00,PrestigiousMotor7840,0,It sure would be nice to watch some huge gains...,1616987439,https://www.reddit.com/r/wallstreetbets/commen...
160,mfghqa,Naked puts,Discussion,1,0.99,BipolarSeacrets,0,So I’m debating on going full reeeee and selli...,1616987552,https://www.reddit.com/r/wallstreetbets/commen...
